In [1]:
%matplotlib inline


# MaD DiGait Pipeline

This pipeline showcases the current gait analysis pipeline used by the MaD-Lab with all required steps:
Preprocessing -> Stride Segmentation -> Event Detection -> Trajectory Reconstruction -> Parameter Estimation

This should serve as a compact example that can be copied and pasted into new projects.
For more details on the individual steps have a look at the extended examples and the documentation of the main classes:

- `Preprocessing <example_preprocessing>`
- `Stride Segmentation (BarthDTW) <example_barth_stride_segmentation>`
- `Event Detection (RamppEventDetection) <example_rampp_event_detection>`
- `Trajectory Reconstruction (double Integration) <example_preprocessing>`
- `Temporal Parameters <example_temporal_parameters>` and `Spatial Parameters <example_spatial_parameters>`


In [2]:
pip install gaitmap

Note: you may need to restart the kernel to use updated packages.


## Load example data



In [76]:
import numpy as np
import pandas as pd 

np.random.seed(0)

example_dataset = pd.read_csv('10may.csv', index_col=False)
sampling_rate_hz = 10

from gaitmap.preprocessing import sensor_alignment

In [77]:
type(example_dataset)

pandas.core.frame.DataFrame

In [78]:
example_dataset.head(1)

,Time,Device name设备名称,Acceleration X(g),Acceleration Y(g),Acceleration Z(g),Angular velocity X(°/s),Angular velocity Y(°/s),Angular velocity Z(°/s),Angle X(°),Angle Y(°),Angle Z(°),Magnetic field X(ʯt),Magnetic field Y(ʯt),Magnetic field Z(ʯt),Temperature(℃),Quaternions 0(),Quaternions 1(),Quaternions 2(),Quaternions 3()
0,14:50:07.533,WT901BLE68(d8:09:5a:ca:b6:e0),0.92,0.898,0.998,0.366,-1.343,-1.526,121.646,-9.932,-90.483,-55.762,39.69,45.178,30.14,0.70258,0.07791,-0.41113,-0.57547


In [79]:
renamed = {'Device name设备名称':'Device name','Acceleration X(g)':'acc_x','Acceleration Y(g)':'acc_y','Acceleration Z(g)':'acc_z',
'Angular velocity X(°/s)':'gyr_x','Angular velocity Y(°/s)':'gyr_y','Angular velocity Z(°/s)':'gyr_z'}

example_dataset = example_dataset.rename(columns=renamed)
example_dataset.head()

,Time,Device name,acc_x,acc_y,acc_z,gyr_x,gyr_y,gyr_z,Angle X(°),Angle Y(°),Angle Z(°),Magnetic field X(ʯt),Magnetic field Y(ʯt),Magnetic field Z(ʯt),Temperature(℃),Quaternions 0(),Quaternions 1(),Quaternions 2(),Quaternions 3()
0,14:50:07.533,WT901BLE68(d8:09:5a:ca:b6:e0),0.920,0.898,0.998,0.366,-1.343,-1.526,121.646,-9.932,-90.483,-55.762,39.690,45.178,30.14,0.70258,0.07791,-0.41113,-0.57547
1,14:50:07.535,WT901BLE68(cc:c4:7a:a1:fe:ea),0.979,-0.101,0.146,0.000,0.977,0.244,-35.090,-79.937,-142.844,-13.132,-37.828,0.294,31.38,-0.05573,0.66525,-0.02762,0.74396
2,14:50:07.537,WT901BLE68(d8:09:5a:ca:b6:e0),0.918,0.906,0.995,1.587,-0.305,-2.258,121.855,-10.107,-90.703,-55.762,39.690,45.178,30.14,0.70258,0.07791,-0.41113,-0.57547
3,14:50:07.544,WT901BLE68(d8:09:5a:ca:b6:e0),0.922,0.908,0.995,1.831,-0.061,-3.479,122.151,-10.135,-90.945,-55.762,39.690,45.178,30.14,0.70258,0.07791,-0.41113,-0.57547
4,14:50:07.548,WT901BLE68(d8:09:5a:ca:b6:e0),0.922,0.911,0.999,-2.991,-0.610,-4.089,122.371,-10.091,-91.214,-55.762,39.690,45.178,30.14,0.70258,0.07791,-0.41113,-0.57547


In [80]:
example_dataset.drop(['Angle X(°)','Angle Y(°)','Angle Z(°)','Magnetic field X(ʯt)','Magnetic field Y(ʯt)','Magnetic field Z(ʯt)','Temperature(℃)','Quaternions 0()','Quaternions 1()','Quaternions 2()','Quaternions 3()'],axis=1,inplace=True)
example_dataset

,Time,Device name,acc_x,acc_y,acc_z,gyr_x,gyr_y,gyr_z
0,14:50:07.533,WT901BLE68(d8:09:5a:ca:b6:e0),0.920,0.898,0.998,0.366,-1.343,-1.526
1,14:50:07.535,WT901BLE68(cc:c4:7a:a1:fe:ea),0.979,-0.101,0.146,0.000,0.977,0.244
2,14:50:07.537,WT901BLE68(d8:09:5a:ca:b6:e0),0.918,0.906,0.995,1.587,-0.305,-2.258
3,14:50:07.544,WT901BLE68(d8:09:5a:ca:b6:e0),0.922,0.908,0.995,1.831,-0.061,-3.479
4,14:50:07.548,WT901BLE68(d8:09:5a:ca:b6:e0),0.922,0.911,0.999,-2.991,-0.610,-4.089
...,...,...,...,...,...,...,...,...
235,14:50:19.112,WT901BLE68(cc:c4:7a:a1:fe:ea),0.967,-0.112,0.143,-0.305,-3.601,2.197
236,14:50:19.202,WT901BLE68(cc:c4:7a:a1:fe:ea),0.986,-0.097,0.163,0.244,-2.258,1.404
237,14:50:19.322,WT901BLE68(cc:c4:7a:a1:fe:ea),0.992,-0.115,0.119,0.854,-0.061,-0.305
238,14:50:19.411,WT901BLE68(cc:c4:7a:a1:fe:ea),0.971,-0.115,0.116,0.061,12.024,-4.089


In [81]:
example_dataset.columns

Index(['Time', 'Device name', 'acc_x', 'acc_y', 'acc_z', 'gyr_x', 'gyr_y',
       'gyr_z'],
      dtype='object')

In [82]:
# Function to convert DataFrame to dictionary of DataFrames
def df_to_dict_of_dfs(df):
    device_names = df['Device name'].unique()
    dfs_dict = {}

    for device_name in device_names:
        device_df = df[df['Device name'] == device_name].copy()
        device_df.drop(columns='Device name', inplace=True)
        dfs_dict[device_name] = device_df.reset_index(drop=True)
    
    return dfs_dict

# Convert DataFrame to dictionary of DataFrames
dfs_dict = df_to_dict_of_dfs(example_dataset)
dfs_dict

{'WT901BLE68(d8:09:5a:ca:b6:e0)':               Time  acc_x  acc_y  acc_z  gyr_x  gyr_y  gyr_z
 0     14:50:07.533  0.920  0.898  0.998  0.366 -1.343 -1.526
 1     14:50:07.537  0.918  0.906  0.995  1.587 -0.305 -2.258
 2     14:50:07.544  0.922  0.908  0.995  1.831 -0.061 -3.479
 3     14:50:07.548  0.922  0.911  0.999 -2.991 -0.610 -4.089
 4     14:50:07.556  0.921  0.921  0.997  0.305  0.366 -1.648
 ..             ...    ...    ...    ...    ...    ...    ...
 115   14:50:18.629  0.850  0.855  0.734  7.080  5.737  2.869
 116   14:50:18.751  0.854  0.860  0.745  8.728  3.906  9.460
 117   14:50:18.843  0.876  0.815  0.747  4.639  4.822  0.427
 118   14:50:18.959  0.882  0.843  0.754  0.366  3.723 -2.625
 119   14:50:19.051  0.871  0.847  0.759 -1.221  1.099 -0.305
 
 [120 rows x 7 columns],
 'WT901BLE68(cc:c4:7a:a1:fe:ea)':               Time  acc_x  acc_y  acc_z  gyr_x   gyr_y  gyr_z
 0     14:50:07.535  0.979 -0.101  0.146  0.000   0.977  0.244
 1     14:50:07.652  0.981 -0.100  0.

In [83]:

# Print the dictionary of DataFrames
for device_name, device_df in dfs_dict.items():
    print(f"Device Name: {device_name}")
    print(device_df)
    print()

Device Name: WT901BLE68(d8:09:5a:ca:b6:e0)
              Time  acc_x  acc_y  acc_z  gyr_x  gyr_y  gyr_z
0     14:50:07.533  0.920  0.898  0.998  0.366 -1.343 -1.526
1     14:50:07.537  0.918  0.906  0.995  1.587 -0.305 -2.258
2     14:50:07.544  0.922  0.908  0.995  1.831 -0.061 -3.479
3     14:50:07.548  0.922  0.911  0.999 -2.991 -0.610 -4.089
4     14:50:07.556  0.921  0.921  0.997  0.305  0.366 -1.648
..             ...    ...    ...    ...    ...    ...    ...
115   14:50:18.629  0.850  0.855  0.734  7.080  5.737  2.869
116   14:50:18.751  0.854  0.860  0.745  8.728  3.906  9.460
117   14:50:18.843  0.876  0.815  0.747  4.639  4.822  0.427
118   14:50:18.959  0.882  0.843  0.754  0.366  3.723 -2.625
119   14:50:19.051  0.871  0.847  0.759 -1.221  1.099 -0.305

[120 rows x 7 columns]

Device Name: WT901BLE68(cc:c4:7a:a1:fe:ea)
              Time  acc_x  acc_y  acc_z  gyr_x   gyr_y  gyr_z
0     14:50:07.535  0.979 -0.101  0.146  0.000   0.977  0.244
1     14:50:07.652  0.981 -0.100 

In [84]:
from gaitmap.utils.datatype_helper import get_multi_sensor_names
get_multi_sensor_names(dfs_dict)

dict_keys(['WT901BLE68(d8:09:5a:ca:b6:e0)', 'WT901BLE68(cc:c4:7a:a1:fe:ea)'])

In [85]:
from gaitmap.utils.datatype_helper import is_sensor_data

is_sensor_data(dfs_dict, frame="sensor")


'multi'

In [86]:
is_sensor_data(dfs_dict['WT901BLE68(d8:09:5a:ca:b6:e0)'], frame="sensor")

'single'

## Preprocessing
Fix the alignment between the sensor coordinate system and the gaitmap coordinate system.
This will be different for each sensor position and recording.



In [87]:
from scipy.spatial.transform import Rotation

# For multiple sensors, we write down the rotation matrices for each sensor into a dict
rotation_matrices = {
      'WT901BLE68(d8:09:5a:ca:b6:e0)': Rotation.from_matrix(np.array([[ 0, 1,  0], [ 0,  0,  1], [1,  0,  0]])),
      'WT901BLE68(cc:c4:7a:a1:fe:ea)': Rotation.from_matrix(np.array([[ 0,  -1,  0], [ 0,  0, -1], [1,  0,  0]]))
}
from gaitmap.utils.rotations import rotate_dataset

# We assume `data` has two sensors with the same names as in the dict above
data = rotate_dataset(dfs_dict, rotation_matrices)
data

{'WT901BLE68(d8:09:5a:ca:b6:e0)':               Time  acc_x  acc_y  acc_z  gyr_x  gyr_y  gyr_z
 0     14:50:07.533  0.898  0.998  0.920 -1.343 -1.526  0.366
 1     14:50:07.537  0.906  0.995  0.918 -0.305 -2.258  1.587
 2     14:50:07.544  0.908  0.995  0.922 -0.061 -3.479  1.831
 3     14:50:07.548  0.911  0.999  0.922 -0.610 -4.089 -2.991
 4     14:50:07.556  0.921  0.997  0.921  0.366 -1.648  0.305
 ..             ...    ...    ...    ...    ...    ...    ...
 115   14:50:18.629  0.855  0.734  0.850  5.737  2.869  7.080
 116   14:50:18.751  0.860  0.745  0.854  3.906  9.460  8.728
 117   14:50:18.843  0.815  0.747  0.876  4.822  0.427  4.639
 118   14:50:18.959  0.843  0.754  0.882  3.723 -2.625  0.366
 119   14:50:19.051  0.847  0.759  0.871  1.099 -0.305 -1.221
 
 [120 rows x 7 columns],
 'WT901BLE68(cc:c4:7a:a1:fe:ea)':               Time  acc_x  acc_y  acc_z   gyr_x  gyr_y  gyr_z
 0     14:50:07.535  0.101 -0.146  0.979  -0.977 -0.244  0.000
 1     14:50:07.652  0.100 -0.145  0.

In [88]:

from gaitmap.preprocessing import sensor_alignment
data = sensor_alignment.align_dataset_to_gravity(data, sampling_rate_hz)
data

{'WT901BLE68(d8:09:5a:ca:b6:e0)':               Time     acc_x     acc_y     acc_z     gyr_x     gyr_y  \
 0     14:50:07.533 -0.023982  0.009595  1.627311 -0.940712 -1.094730   
 1     14:50:07.537 -0.015831  0.006758  1.628892 -0.644169 -2.621603   
 2     14:50:07.544 -0.016499  0.003897  1.632264 -0.320267 -3.756945   
 3     14:50:07.548 -0.014989  0.006300  1.636381  2.100361 -1.183376   
 4     14:50:07.556 -0.006030  0.003184  1.640257  0.480090 -1.525691   
 ..             ...       ...       ...       ...       ...       ...   
 115   14:50:18.629  0.039209 -0.140563  1.403918 -0.063811 -3.349719   
 116   14:50:18.751  0.038517 -0.135665  1.415646 -3.895905  1.096023   
 117   14:50:18.843 -0.010148 -0.137595  1.403772  1.122263 -3.539277   
 118   14:50:18.959  0.007204 -0.142009  1.427190  3.330450 -3.045830   
 119   14:50:19.051  0.015504 -0.132400  1.426310  1.630843  0.265159   
 
          gyr_z  
 0    -1.477467  
 1    -0.649924  
 2    -1.114681  
 3    -4.497355  

## Stride Segmentation
In this step the continuous datastream is segmented into individual strides.
For longer datasets it might be required to first identify segments of walking to reduce the chance of
false-positives.



In [89]:
from gaitmap.stride_segmentation import BarthDtw
from gaitmap.utils.coordinate_conversion import convert_to_fbf

dtw = BarthDtw()
# Convert data to foot-frame
bf_data = convert_to_fbf(data, left_like="WT901BLE68(d8:09:5a:ca:b6:e0)", right_like="WT901BLE68(cc:c4:7a:a1:fe:ea)")
dtw = dtw.segment(data=bf_data, sampling_rate_hz=sampling_rate_hz)

In [90]:
segmented_strides = dtw.stride_list_
segmented_strides

{'WT901BLE68(d8:09:5a:ca:b6:e0)':       start  end
 s_id            
 0         0   10
 1        20   28
 2        38   48
 3        48   55
 4        56   65
 5        73   84
 6        95  103,
 'WT901BLE68(cc:c4:7a:a1:fe:ea)':       start  end
 s_id            
 0         2   14
 1        14   23
 2        23   35
 3        35   53
 4        53   72
 5        72   91
 6        91   98}

In [91]:
dtw_warping_path = dtw.paths_
dtw_warping_path

{'WT901BLE68(d8:09:5a:ca:b6:e0)': [array([[ 0,  0],
         [ 1,  1],
         [ 2,  2],
         [ 3,  3],
         [ 4,  3],
         [ 5,  4],
         [ 5,  5],
         [ 5,  6],
         [ 5,  7],
         [ 5,  8],
         [ 5,  9],
         [ 6, 10],
         [ 7, 10],
         [ 8, 10]]),
  array([[ 0, 20],
         [ 1, 21],
         [ 2, 22],
         [ 3, 23],
         [ 4, 24],
         [ 5, 25],
         [ 5, 26],
         [ 6, 27],
         [ 7, 28],
         [ 8, 28]]),
  array([[ 0, 39],
         [ 1, 40],
         [ 2, 41],
         [ 3, 42],
         [ 4, 43],
         [ 5, 44],
         [ 6, 45],
         [ 7, 46],
         [ 8, 47]]),
  array([[ 0, 48],
         [ 1, 49],
         [ 2, 49],
         [ 3, 50],
         [ 4, 51],
         [ 5, 52],
         [ 6, 53],
         [ 7, 54],
         [ 8, 54]]),
  array([[ 0, 57],
         [ 1, 58],
         [ 2, 58],
         [ 3, 59],
         [ 4, 60],
         [ 5, 61],
         [ 6, 62],
         [ 7, 63],
         

In [92]:
bf_data

{'WT901BLE68(d8:09:5a:ca:b6:e0)':        acc_pa    acc_ml    acc_si    gyr_pa    gyr_ml     gyr_si
 0   -0.023982  0.009595 -1.627311  0.940712  1.094730   1.477467
 1   -0.015831  0.006758 -1.628892  0.644169  2.621603   0.649924
 2   -0.016499  0.003897 -1.632264  0.320267  3.756945   1.114681
 3   -0.014989  0.006300 -1.636381 -2.100361  1.183376   4.497355
 4   -0.006030  0.003184 -1.640257 -0.480090  1.525691   0.620215
 ..        ...       ...       ...       ...       ...        ...
 115  0.039209 -0.140563 -1.403918  0.063811  3.349719  -8.946845
 116  0.038517 -0.135665 -1.415646  3.895905 -1.096023 -12.827591
 117 -0.010148 -0.137595 -1.403772 -1.122263  3.539277  -5.582865
 118  0.007204 -0.142009 -1.427190 -3.330450  3.045830  -0.718561
 119  0.015504 -0.132400 -1.426310 -1.630843 -0.265159   0.248410
 
 [120 rows x 6 columns],
 'WT901BLE68(cc:c4:7a:a1:fe:ea)':        acc_pa    acc_ml    acc_si     gyr_pa    gyr_ml    gyr_si
 0    0.007012 -0.005980 -0.994924  -0.974380  0.

In [93]:
dtw

BarthDtw(conflict_resolution=True, find_matches_method='find_peaks', max_cost=4.0, max_match_length_s=3.0, max_signal_stretch_ms=None, max_template_stretch_ms=None, memory=None, min_match_length_s=0.6, resample_template=True, snap_to_min_axis='gyr_ml', snap_to_min_win_ms=300, template=BarthOriginalTemplate(scaling=FixedScaler(offset=0, scale=500.0), use_cols=None))

## Event detection
For each identified stride, we now identify important stride events.



In [94]:
from gaitmap.event_detection import RamppEventDetection

ed = RamppEventDetection()
ed = ed.detect(data=bf_data, stride_list=dtw.stride_list_, sampling_rate_hz=sampling_rate_hz)

ValueError: The chosen values are smaller than the sample time (100.0 ms)

## Trajectory Reconstruction
Using the identified events the trajectory of each stride is reconstructed using double integration starting from the
`min_vel` event of each stride.



In [ ]:
from gaitmap.trajectory_reconstruction import StrideLevelTrajectory

trajectory = StrideLevelTrajectory()
trajectory = trajectory.estimate(
    data=data, stride_event_list=ed.min_vel_event_list_, sampling_rate_hz=sampling_rate_hz
)

## Temporal Parameter Calculation
Now we have all information to calculate relevant temporal parameters (like stride time)



In [ ]:
from gaitmap.parameters import TemporalParameterCalculation

temporal_paras = TemporalParameterCalculation()
temporal_paras = temporal_paras.calculate(stride_event_list=ed.min_vel_event_list_, sampling_rate_hz=sampling_rate_hz)

## Spatial Parameter Calculation
Like the temporal parameters, we can also calculate the spatial parameter.



In [ ]:
from gaitmap.parameters import SpatialParameterCalculation

spatial_paras = SpatialParameterCalculation()
spatial_paras = spatial_paras.calculate(
    stride_event_list=ed.min_vel_event_list_,
    positions=trajectory.position_,
    orientations=trajectory.orientation_,
    sampling_rate_hz=sampling_rate_hz,
)

## Inspecting the Results
The class of each step allows you to inspect all results in detail.
Here we will just print and plot the most important once.
Note, that the plots below are for sure not the best way to represent results!



In [ ]:
import matplotlib.pyplot as plt

print(
    f"The following number of strides were identified and parameterized for each sensor: {({k: len(v) for k, v in ed.min_vel_event_list_.items()})}"
)

The following number of strides were identified and parameterized for each sensor: {'WT901BLE68(d8:09:5a:ca:b6:e0)': 0, 'WT901BLE68(cc:c4:7a:a1:fe:ea)': 0}


In [ ]:
for k, v in temporal_paras.parameters_pretty_.items():
    v.plot()
    plt.title(f"All temporal parameters of sensor {k}")

TypeError: no numeric data to plot

In [ ]:
for k, v in spatial_paras.parameters_pretty_.items():
    v[["stride length [m]", "gait velocity [m/s]", "arc length [m]"]].plot()
    plt.title(f"All spatial parameters of sensor {k}")

In [ ]:
for k, v in spatial_paras.parameters_pretty_.items():
    v.filter(like="angle").plot()
    plt.title(f"All angle parameters of sensor {k}")